## BIGRAM TRANSFORMER MODEL

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cpu


In [2]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(len(text), text[0:50])    # print the text length and first 50 characters of text

232309 ﻿  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK 


In [3]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


### TOKENIZERS  --- encoder/decoder model using Pytorch tensors

In [4]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [5]:
# Test the tokenizer

encode_hello = encode('hello')
decode_hello = decode(encode_hello)

print(encode_hello) 
print(decode_hello)

[61, 58, 65, 65, 68]
hello


In [6]:
# Tokenizer with Pytorch tensors
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [12]:
torch.randint(len(data) - block_size, (batch_size,))

tensor([ 45463, 120591,  81414,  78599])

### Validation and Training Splits

In [18]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[62, 67, 60,  1, 72, 68, 66, 58],
        [67, 73, 58, 57,  1, 73, 61, 58],
        [67, 72, 73, 54, 67, 73, 65, 78],
        [ 1, 78, 68, 74,  1, 54, 67, 57]])
targets:
tensor([[67, 60,  1, 72, 68, 66, 58,  1],
        [73, 58, 57,  1, 73, 61, 58,  1],
        [72, 73, 54, 67, 73, 65, 78,  1],
        [78, 68, 74,  1, 54, 67, 57,  1]])


* Gradient descent(reduce loss when prediciting next token) 

In [19]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

* nn module subclass

In [20]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


Wb6a:Z'Mk9*Gl-FxE]'cl﻿Lf52Pc1??uV5Xz6W8dh"_uGj1Q,yit.1U 50Xil66"_DrKnO)UVlW! ZvEX:L7uG?jY MK[L"mMkT5(f(?Mk_jNe6 Z0WFye'619xg
tf-&'ywTljO7DkJ4G._*:pv,KU62y[QH:LRoy
Y7zOa﻿n&Zg[QmJMwm﻿U);XDw6W4;CmZz.rbpAEt0"_f-Z)AERQ?
xhHePa?﻿yBS6iYaKG*BR0oK.q;HFHtO8BAgp."_bcVvwJwblclh-9(osEUSk8l&]y_[2ye7x-(Hj1q5!9iNI"rPnKOonpUHwBd-6)vZs*J)uI']6g8KLq;7&]CMm]6B3d_njYoF12?
2v.HLcQS!Qby4pcOW-w8e!& hn_"qB Ip5JFIpd8Iqe76Ia]C2UkpdLZ﻿pUBA.(kBF?_u*c_DqIkJM﻿c4BR8eJMHncVyeRSQ:!xu)& -3PT(-t.8j ﻿)GJ'-ZUU)U!6NjI"Df_5nI_NeD79xPG


In [21]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.928, val loss: 4.925
step: 250, train loss: 4.892, val loss: 4.867
step: 500, train loss: 4.800, val loss: 4.784
step: 750, train loss: 4.736, val loss: 4.735
4.520847320556641


In [22]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


t﻿Bjtt.8G Y"qZh-G1:cpe:R*8M[AOYfzM*4J)aKBon!3E7!O[z9p[V*GlgcjY(I4mT﻿﻿u;08Vw*jBAo6Ykhe6Q;ROqUuN_DkN'QCUiQ'hFKCRtHBDsnvmSikM(mn]:"S;JI, wp-lQbvW5'TS 
DH_"w)YoJv,Psp(
gcIkj
nk& 6z'8d85Y-kShEK
[X;?nJ935"w8cjCIeLB)J!(KVwmTTL1(5k[S1mWYa.*Gz.B4TSNg(?*zXqG12FK:yenr(W5vkO_Z07XUub6Jn(oe6yBFK2Y79xaXqk5BZ 9erw_-?jNA G;C ax0mtrG
p0a.e?_
6dk﻿s&Hehy&-[ tt.qeOFK[w8Mgs*blH0ImwS,zP8dIf'x_-([1tCFiwp!Oz6N3UcOYI.rtC9Rx﻿x-Y3D1VXL;Cm[]&JAk8)rXAP.w89DrpBvPoJMdmi"cZ-BXL﻿'iyrHA"o!Z'JR CY
QGQS.17&]
qBKx2RUhMHSZa0P
PFxE;B6
